In [1]:
import syft as sy
duet = sy.join_duet(loopback=True)


🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/tk/Desktop/Studia/masters/env/lib/python3.10/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [2]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [3]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: a145c5f08cc84df38819fbde95b5c364>,[DO1 data],6 samples,<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 8909416d277d462f97f35ffa4b5761cd>,[DO2 data],5 samples,<class 'torch.Tensor'>


In [6]:
data1_ptr = duet.store[0]
data2_ptr = duet2.store[0]

print(data1_ptr)
print(data2_ptr)

In [7]:
import torch

In [8]:
in_dim = 1
out_dim = 1

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)
    
    def forward(self, x):
        x = self.linear(x)
        return x

In [10]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):
    
    losses = []
    
    for i in range(iterations):
        optim.zero_grad()
        output = model(data_ptr)
        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        loss_item = loss.item()
        try:
            loss_value = loss_item.get(
                reason="To eval training",
                request_block=True,
                timeout_secs=5
            )
        except Exception as e:
            print(e)
        print("Epoch",i,"loss",loss_value)
        losses.append(loss_value)
        loss.backward()
        optim.step()
    return losses

### Each data owner will have a model and the training will be performed remotely

In [11]:
import torch as th
import numpy as np

In [12]:
base_model = SyNet(torch)

In [13]:
remote_model1 = base_model.send(duet)

In [14]:
remote_torch1 = duet.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

### Dummy data to check model

In [15]:
target1_ptr = th.FloatTensor(np.array([5,10,15,22,33,38]).reshape(-1,1))
target1_ptr

tensor([[ 5.],
        [10.],
        [15.],
        [22.],
        [33.],
        [38.]])

In [16]:
iteration = 10
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr,target1_ptr)

Epoch 0 loss 52.02812576293945
Epoch 1 loss 14.859127044677734
Epoch 2 loss 2.7746779918670654
Epoch 3 loss 8.565349578857422
Epoch 4 loss 17.505184173583984
Epoch 5 loss 19.677640914916992
Epoch 6 loss 15.440752983093262
Epoch 7 loss 9.011955261230469
Epoch 8 loss 4.1528000831604
Epoch 9 loss 2.755591630935669


In [17]:
remote_model2 = base_model.send(duet2)

In [18]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

In [19]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

tensor([[35.],
        [40.],
        [45.],
        [55.],
        [60.]])

In [20]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 200.14926147460938
Epoch 1 loss 55.6126594543457
Epoch 2 loss 15.525152206420898
Epoch 3 loss 44.60530471801758
Epoch 4 loss 77.37857818603516
Epoch 5 loss 79.2303237915039
Epoch 6 loss 58.002235412597656
Epoch 7 loss 32.604637145996094
Epoch 8 loss 17.29300308227539
Epoch 9 loss 17.08054542541504


### Averaging

In [21]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print(base_model.parameters())
print(param1)
print(param2)

[Parameter containing:
tensor([[0.4731]], requires_grad=True), Parameter containing:
tensor([0.3195], requires_grad=True)]
[Parameter containing:
tensor([[0.6345]], requires_grad=True), Parameter containing:
tensor([0.4473], requires_grad=True)]
[Parameter containing:
tensor([[0.6131]], requires_grad=True), Parameter containing:
tensor([0.4207], requires_grad=True)]


In [22]:
remote_model1_updates = remote_model1.get(
    request_block=True).state_dict()

print(remote_model1_updates)

OrderedDict([('linear.weight', tensor([[0.6345]])), ('linear.bias', tensor([0.4473]))])


In [23]:
remote_model2_updates = remote_model2.get(
    request_block=True).state_dict()

print(remote_model2_updates)

OrderedDict([('linear.weight', tensor([[0.6131]])), ('linear.bias', tensor([0.4207]))])


In [24]:
from collections import OrderedDict

##### Now calculate the average of the models

In [25]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[0.6238]])), ('linear.bias', tensor([0.4340]))])


### Load avg params

In [26]:
combined_model = SyNet(torch)

In [27]:
combined_model.load_state_dict(avg_updates)

In [28]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [29]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [30]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 11.038990020751953 Ground Truth: 12.0
Prediction: 16.029573440551758 Ground Truth: 15.0
Prediction: 20.396333694458008 Ground Truth: 20.0
Prediction: 31.625146865844727 Ground Truth: 30.0
Prediction: 50.33983612060547 Ground Truth: 50.0


In [31]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 8.361913681030273 Ground Truth: 12.0
Prediction: 12.146585464477539 Ground Truth: 15.0
Prediction: 15.458173751831055 Ground Truth: 20.0
Prediction: 23.97368812561035 Ground Truth: 30.0
Prediction: 38.16621017456055 Ground Truth: 50.0


# Centralised Model

In [32]:
data2_ptr

In [33]:
data1_ptr

In [40]:
data = th.cat((data1_ptr, data2_ptr),0)

TypeError: expected Tensor as element 0 in argument 0, but got TensorPointer

In [41]:
data

In [38]:
target = th.cat((target1_ptr, target2_ptr),0)

In [39]:
target

tensor([[ 5.],
        [10.],
        [15.],
        [22.],
        [33.],
        [38.],
        [35.],
        [40.],
        [45.],
        [55.],
        [60.]])